### Caavo Computer Vision Challenge

#### Loading relevant libraries

In [38]:
import pandas as pd
import os, shutil
import numpy as np
from keras.models import Sequential
from keras.utils import np_utils
from keras import layers, models, regularizers, optimizers
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image
from keras.applications import Xception

In [2]:
from keras import backend
import importlib 
print(backend._BACKEND)
from keras import backend as K
def set_keras_backend(backend):
    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        importlib.reload(K)
        assert K.backend() == backend
print ("Change Keras Backend to Tensorflow")        
set_keras_backend("tensorflow")  
from keras import backend
print(backend._BACKEND)

tensorflow
Change Keras Backend to Tensorflow
tensorflow


#### Dividing train dataset into train, test and validation dataset (Ratio 60:20:20)

In [3]:
src_dir = 'data/dataset/train/'
trgt_train_dir = 'train_set/'
trgt_test_dir = 'test_set/'
trgt_val_dir = 'val_set/'
sub_fldr = [str(i) for i in range(15)]

In [4]:
if not os.path.exists(trgt_train_dir):
    os.mkdir(trgt_train_dir)
if not os.path.exists(trgt_test_dir):
    os.mkdir(trgt_test_dir)
if not os.path.exists(trgt_val_dir):
    os.mkdir(trgt_val_dir)

for i in sub_fldr:
    trgt_sub_train_dir = 'train_set/' + i
    trgt_sub_test_dir = 'test_set/' + i
    trgt_sub_val_dir = 'val_set/' + i
    
    if not os.path.exists(trgt_sub_train_dir):
        os.mkdir(trgt_sub_train_dir)
    if not os.path.exists(trgt_sub_test_dir):
        os.mkdir(trgt_sub_test_dir)
    if not os.path.exists(trgt_sub_val_dir):
        os.mkdir(trgt_sub_val_dir)

In [5]:
for i in sub_fldr:
    src_sub_dir = src_dir + i
    src_files = list(os.listdir(src_sub_dir))
    
    trgt_sub_train_dir = 'train_set/' + i
    trgt_sub_test_dir = 'test_set/' + i
    trgt_sub_val_dir = 'val_set/' + i
    
    train_part = src_files[:int(0.8*len(src_files))]
    test_part = src_files[int(0.8*len(src_files)):int(0.9*len(src_files))]
    val_part = src_files[int(0.9*len(src_files)):]
    
    for filename in train_part:
        shutil.copy(src_sub_dir+'/'+filename,trgt_sub_train_dir+'/'+filename)
    
    for filename in test_part:
        shutil.copy(src_sub_dir+'/'+filename,trgt_sub_test_dir+'/'+filename)
        
    for filename in val_part:
        shutil.copy(src_sub_dir+'/'+filename,trgt_sub_val_dir+'/'+filename)
        
    print('Subfolder '+ i +' done')    

Subfolder 0 done
Subfolder 1 done
Subfolder 2 done
Subfolder 3 done
Subfolder 4 done
Subfolder 5 done
Subfolder 6 done
Subfolder 7 done
Subfolder 8 done
Subfolder 9 done
Subfolder 10 done
Subfolder 11 done
Subfolder 12 done
Subfolder 13 done
Subfolder 14 done


#### Using pretrained CNN (Xception) 

In [41]:
conv_base = Xception(weights='imagenet',include_top=False,input_shape=(150, 150, 3))
conv_base.trainable = False

learning_rate = 0.001
decay_rate = learning_rate / epochs
rmsp = optimizers.RMSprop(lr=learning_rate, decay=decay_rate)

model = Sequential()
model.add(conv_base)
model.add(Dropout(0.5))
model.add(layers.Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(15, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=rmsp,metrics=['categorical_accuracy'])

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
activation_9 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 150, 150, 32)      128       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 150, 150, 32)      9248      
_________________________________________________________________
activation_10 (Activation)   (None, 150, 150, 32)      0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 75, 75, 32)        0         
__________

In [43]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [44]:
### Parameters
batch_size = 64
epochs=100
###

train_generator = train_datagen.flow_from_directory(
    trgt_train_dir,
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    trgt_val_dir,
    target_size=(150,150),
    batch_size=batch_size,
    class_mode='categorical')

Found 37347 images belonging to 15 classes.
Found 12458 images belonging to 15 classes.


In [45]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=1000,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=100)

Epoch 1/25
  3/100 [..............................] - ETA: 53:50 - loss: 11.0362 - categorical_accuracy: 0.0833 

KeyboardInterrupt: 

In [ ]:
model.save('caavo_main_v2_85.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()